# magic-growth

This is the main notebook/source code for now, to be reorganized later. Code blocks are organized roughly in the order used, with two main function definition blocks.  There are also separate code blocks for generating data for different states for ease of working.

In [74]:
import numpy as np
import math
import random
from scipy.stats import unitary_group as ug
#from sympy.ntheory import isprime

In [22]:
#useful number theory definitions and helper functions
def inv2(k):
    if (k % 2 == 0):
        print("Invalid input:",k,"is not odd")
        return -1
    return int((k+1)/2)

def summatrix(m):
    return sum([sum(m[i]) for i in range(len(m))])

def kronaction(m,i,n,d):
    for j in range(i):
        m = np.kron(np.identity(d),m)
    for j in range(i+1,n):
        m = np.kron(m, np.identity(d))
    return m

def basis_to_int(string,n,d):
    if (n != len(string)):
        print("Invalid input: string must contain",n,"qudits")
        return -1
    return sum([int(string[i])*d**(n-i-1) for i in range(n)])

def int_to_basis(index,n,d):
    result = ""
    for i in range(n):
        result += str(index // d**(n-i-1))
        index %= d**(n-i-1)
    return result

In [72]:
#generate random pure states with local dimension d and size n
def random_pure_state(n,d):
    vec = np.random.normal(d**n)
    norm = np.linalg.norm(vec)
    phases = np.exp(np.random.rand(d**n)*2j*np.pi)
    return vec*phases/norm

#generate random mixed states with local dimension d and size n as statistical mixtures of up to j pure states
def random_mixed_state(n,d,j):
    #TODO
    return 0

In [24]:
#discrete wigner function calculation
def x_shift(vec):
    return np.append(vec[1:],vec[0])

def z_clock(vec):
    return vec * np.exp(2j*np.pi/len(vec)*np.array([i for i in range(len(vec))]))

def weyl(vec, a_1, a_2):
    #TODO
    return 0

def weyl_dag(vec, a_1, a_2):
    #TODO
    return 0

def phase_sp_pt_0(vec, a_1, a_2):
    #TODO
    return 0

def phase_sp_pt(vec, a_1, a_2):
    #TODO
    return 0

def discrete_wigner_pure(vec, p, q):
    d = len(vec)
    if (d % 2 == 0):
        print("Invalid input:",d,"is not odd")
        return -1
    result = 0
    inv2d = inv2(d)
    for i in range(d):
        result += 1/d * (np.e ** (-2j*np.pi*p*i/d)) * np.conj(vec[(q - inv2d*i) % d]) * vec[(q + inv2d*i) % d]
    #take real part to get rid of weird rounding errors
    return np.real(result)

def discrete_wigner_general(rho, d, p, q):
    #TODO
    return 0

def full_discrete_wigner_pure(vec):
    return [[discrete_wigner_pure(vec, p, q) for q in range(len(vec))] for p in range(len(vec))]

def sn_pure(vec):
    d = len(vec)
    result = 0
    for p in range(d):
        for q in range(d):
            result += abs(discrete_wigner_pure(vec, p, q))
    return (result - 1)/2

In [17]:
#testing
a = random_pure_state(2,3)
print(a)
wabs = [[abs(discrete_wigner_pure(a, p, q)) for q in range(9)] for p in range(9)]
print((summatrix(wabs) - 1)/2)
print(sn_pure(a))

[ 0.56543119-0.42604748j  0.59197074+0.13921831j  0.00323071-0.11622446j
  0.09631194-0.09934678j -0.17497252-0.16623632j -0.15144416-0.08012211j
  0.0026315 +0.0410843j   0.08200108+0.01349595j -0.18339454-0.98299375j]
1.8805352147698264
1.8805352147698255


In [35]:
#construct some states of interest
def random_local_2d(m,n,d,t):
    #TODO
    #return d**(m x n) state vector
    #allows 2-qubit gates between neighboring qubits
    state = np.zeros(d**(m*n))
    return state
    
def random_local_1d(n,d,t):
    #allows 2-qubit gates between neighboring qubits
    #n should be at least 3 to use toffoli gates
    state = np.zeros(d**n)*(1+0j)
    state[random.randrange(d**n)] = 1
    history = []
    g = 5
    if (n < 2):
        g = 3
    elif (n < 3):
        g = 4
    for s in range(t):
        avail = {i:1 for i in range(n)}
        for sp in range(n):
            gt = random.randrange(g)
            #history.append(gt)
            if (gt == 0):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = hadamard(state,i,n,d)
            elif (gt == 1):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = phase(state,i,n,d)
            elif (gt == 2):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = smult(state,i,n,d)
            elif (gt == 3):
                i = random.randrange(n-1)
                if (avail[i] and avail[i+1]):
                    avail[i] = 0
                    avail[i+1] = 0
                    r = random.randrange(1)
                    if (r == 0):
                        j = i + 1
                    else:
                        j = i
                        i += 1
                    state = sumd(state,i,j,n,d)
            else:
                i = random.randrange(n-2)
                if (avail[i] and avail[i+1] and avail[i+2]):
                    avail[i] = 0
                    avail[i+1] = 0
                    avail[i+2] = 0
                    r = random.randrange(5)
                    if (r == 0):
                        j = i + 1
                        k = i + 2
                    elif (r == 1):
                        k = i + 1
                        j = i + 2
                    elif (r == 2):
                        j = i
                        k = i + 2
                        i += 1
                    elif (r == 3):
                        j = i
                        k = i + 1
                        i += 2
                    elif (r == 4):
                        k = i
                        j = i + 2
                        i += 1
                    else:
                        k = i
                        j = i + 1
                        i += 2
                    state = toffoli(state,i,j,k,n,d)
    return state

def random_local_1d_clifford(n,d,t):
    #allows 2-qubit gates between neighboring qubits
    state = np.zeros(d**n)*(1+0j)
    state[random.randrange(d**n)] = 1
    #history = []
    g = 4
    if (n < 2):
        g = 3
    for s in range(t):
        avail = {i:1 for i in range(n)}
        for sp in range(n):
            gt = random.randrange(g)
            #history.append(gt)
            if (gt == 0):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = hadamard(state,i,n,d)
            elif (gt == 1):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = phase(state,i,n,d)
            elif (gt == 2):
                i = random.randrange(n)
                if (avail[i]):
                    avail[i] = 0
                    state = smult(state,i,n,d)
            elif (gt == 3):
                i = random.randrange(n-1)
                if (avail[i] and avail[i+1]):
                    avail[i] = 0
                    avail[i+1] = 0
                    r = random.randrange(1)
                    if (r == 0):
                        j = i + 1
                    else:
                        j = i
                        i += 1
                    state = sumd(state,i,j,n,d)
    #print(history)
    return state
    
#generalization of walsh-hadamard
def hadamard(vec,i,n,d):
    omega = np.e**(2j*np.pi/d)
    h = [[1/math.sqrt(d)*omega**(k*(d-i)) for k in range(d)] for i in range(d)]
    #print(type(vec),vec)
    return np.dot(kronaction(h,i,n,d),vec)

#generalization of phase gate for odd prime d
def phase(vec,i,n,d):
    p = np.identity(d)*(1+0j)
    for f in range(d):
        p[f][f] = np.e**(f*(f-1)*1j*np.pi/d)
    return np.dot(kronaction(p,i,n,d),vec)

#generalization of s gate, takes j -> 2j which works for odd prime d
def smult(vec,i,n,d):
    s = np.zeros((d,d))*(1+0j)
    for f in range(d):
        s[(2*f)%d][f] = 1
    return np.dot(kronaction(s,i,n,d),vec)

#generalization of cnot, i is control and j is target
def sumd(vec,i,j,n,d):
    c = np.zeros((d**n,d**n))
    for k in range(d**n):
        out = int_to_basis(k,n,d)
        b = (int(out[j]) - int(out[i])) % d
        c[k][basis_to_int(out[:j] + str(b) + out[j+1:],n,d)] = 1
    return np.dot(c,vec)

#generalization of toffoli, takes a, b, c -> a, b, c + ab
def toffoli(vec,i,j,k,n,d):
    t = np.zeros((d**n,d**n))*(1+0j)
    for f in range(d**n):
        out = int_to_basis(f,n,d)
        b = (int(out[k]) - int(out[i])*int(out[j])) % d
        t[f][basis_to_int(out[:k] + str(b) + out[k+1:],n,d)] = 1
    return np.dot(t,vec)

In [31]:
#testing
#sumd([0,0,1,0],0,1,2,2)
sumd([0,0,0,3,0,0,1,2,0],0,1,2,3)
hadamard([0,0,1,0],0,2,2)
phase([1,1,1,1,1,1,1,1,1],0,2,3)
smult([0,1,2,3,4],0,1,5)
toffoli([0,0,0,0,0,0,1,0],0,1,2,3,2)

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])

In [69]:
#more testing
#a = random_local_1d_clifford(4,3,20)
#print(np.ndarray.round(a,4))
#print(summatrix(full_discrete_wigner_pure(a)))
#print(sn_pure(a))
b = random_pure_state(4,3)
#print(b)
print(sn_pure(b))
#c = random_local_1d(5,3,20)
#print(np.ndarray.round(c,4))
#print(sn_pure(c))

2.017376035705117


### notes and organization
- Planning to run for n = 3, if time 4, 5, note 6, 7, 8 are very long
- Just focus on d = 3 to save time
- t up to 200

In [77]:
d = 3
for n in [4, 5]:
    for t in [10 + 10*i for i in range(12)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        for sample in range(100):
            print(d, t, sample)
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  

3 10 0
3 10 1
3 10 2
3 10 3
3 10 4
3 10 5
3 10 6
3 10 7
3 10 8
3 10 9
3 10 10
3 10 11
3 10 12
3 10 13
3 10 14
3 10 15
3 10 16
3 10 17
3 10 18
3 10 19
3 10 20
3 10 21
3 10 22
3 10 23
3 10 24
3 10 25
3 10 26
3 10 27
3 10 28
3 10 29
3 10 30
3 10 31
3 10 32
3 10 33
3 10 34
3 10 35
3 10 36
3 10 37
3 10 38
3 10 39
3 10 40
3 10 41
3 10 42
3 10 43
3 10 44
3 10 45
3 10 46
3 10 47
3 10 48
3 10 49
3 10 50
3 10 51
3 10 52
3 10 53
3 10 54
3 10 55
3 10 56
3 10 57
3 10 58
3 10 59
3 10 60
3 10 61
3 10 62
3 10 63
3 10 64
3 10 65
3 10 66
3 10 67
3 10 68
3 10 69
3 10 70
3 10 71
3 10 72
3 10 73
3 10 74
3 10 75
3 10 76
3 10 77
3 10 78
3 10 79
3 10 80
3 10 81
3 10 82
3 10 83
3 10 84
3 10 85
3 10 86
3 10 87
3 10 88
3 10 89
3 10 90
3 10 91
3 10 92
3 10 93
3 10 94
3 10 95
3 10 96
3 10 97
3 10 98
3 10 99
3 20 0
3 20 1
3 20 2
3 20 3
3 20 4
3 20 5
3 20 6
3 20 7
3 20 8
3 20 9
3 20 10
3 20 11
3 20 12
3 20 13
3 20 14
3 20 15
3 20 16
3 20 17
3 20 18
3 20 19
3 20 20
3 20 21
3 20 22
3 20 23
3 20 24
3 20 25
3 20 26
3 20

3 110 23
3 110 24
3 110 25
3 110 26
3 110 27
3 110 28
3 110 29
3 110 30
3 110 31
3 110 32
3 110 33
3 110 34
3 110 35
3 110 36
3 110 37
3 110 38
3 110 39
3 110 40
3 110 41
3 110 42
3 110 43
3 110 44
3 110 45
3 110 46
3 110 47
3 110 48
3 110 49
3 110 50
3 110 51
3 110 52
3 110 53
3 110 54
3 110 55
3 110 56
3 110 57
3 110 58
3 110 59
3 110 60
3 110 61
3 110 62
3 110 63
3 110 64
3 110 65
3 110 66
3 110 67
3 110 68
3 110 69
3 110 70
3 110 71
3 110 72
3 110 73
3 110 74
3 110 75
3 110 76
3 110 77
3 110 78
3 110 79
3 110 80
3 110 81
3 110 82
3 110 83
3 110 84
3 110 85
3 110 86
3 110 87
3 110 88
3 110 89
3 110 90
3 110 91
3 110 92
3 110 93
3 110 94
3 110 95
3 110 96
3 110 97
3 110 98
3 110 99
3 120 0
3 120 1
3 120 2
3 120 3
3 120 4
3 120 5
3 120 6
3 120 7
3 120 8
3 120 9
3 120 10
3 120 11
3 120 12
3 120 13
3 120 14
3 120 15
3 120 16
3 120 17
3 120 18
3 120 19
3 120 20
3 120 21
3 120 22
3 120 23
3 120 24
3 120 25
3 120 26
3 120 27
3 120 28
3 120 29
3 120 30
3 120 31
3 120 32
3 120 33
3 120 34
3 

In [ ]:
n = 3
for d in [5, 7]:
    for t in [10 + 20*i for i in range(10)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        print(n, d, t)
        for sample in range(50):
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  

3 5 10
3 5 30
3 5 50
3 5 70
3 5 90
3 5 110
3 5 130
3 5 150
3 5 170
3 5 190
3 7 10
3 7 30
3 7 50
3 7 70


In [81]:
d = 3
for n in [3]:
    for t in [55 + 5*i for i in range(4)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        print(n, d, t)
        for sample in range(150):
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  
        
for n in [4]:
    for t in [55 + 5*i for i in range(4)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        print(n, d, t)
        for sample in range(150):
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  
        
for n in [5]:
    for t in [75 + 5*i for i in range(4)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        for sample in range(150):
            print(d, t, sample)
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  

3 3 5 0
3 3 5 1
3 3 5 2
3 3 5 3
3 3 5 4
3 3 5 5
3 3 5 6
3 3 5 7
3 3 5 8
3 3 5 9
3 3 5 10
3 3 5 11
3 3 5 12
3 3 5 13
3 3 5 14
3 3 5 15
3 3 5 16
3 3 5 17
3 3 5 18
3 3 5 19
3 3 5 20
3 3 5 21
3 3 5 22
3 3 5 23
3 3 5 24
3 3 5 25
3 3 5 26
3 3 5 27
3 3 5 28
3 3 5 29
3 3 5 30
3 3 5 31
3 3 5 32
3 3 5 33
3 3 5 34
3 3 5 35
3 3 5 36
3 3 5 37
3 3 5 38
3 3 5 39
3 3 5 40
3 3 5 41
3 3 5 42
3 3 5 43
3 3 5 44
3 3 5 45
3 3 5 46
3 3 5 47
3 3 5 48
3 3 5 49
3 3 5 50
3 3 5 51
3 3 5 52
3 3 5 53
3 3 5 54
3 3 5 55
3 3 5 56
3 3 5 57
3 3 5 58
3 3 5 59
3 3 5 60
3 3 5 61
3 3 5 62
3 3 5 63
3 3 5 64
3 3 5 65
3 3 5 66
3 3 5 67
3 3 5 68
3 3 5 69
3 3 5 70
3 3 5 71
3 3 5 72
3 3 5 73
3 3 5 74
3 3 5 75
3 3 5 76
3 3 5 77
3 3 5 78
3 3 5 79
3 3 5 80
3 3 5 81
3 3 5 82
3 3 5 83
3 3 5 84
3 3 5 85
3 3 5 86
3 3 5 87
3 3 5 88
3 3 5 89
3 3 5 90
3 3 5 91
3 3 5 92
3 3 5 93
3 3 5 94
3 3 5 95
3 3 5 96
3 3 5 97
3 3 5 98
3 3 5 99
3 3 5 100
3 3 5 101
3 3 5 102
3 3 5 103
3 3 5 104
3 3 5 105
3 3 5 106
3 3 5 107
3 3 5 108
3 3 5 109
3 3 5 110


3 3 30 68
3 3 30 69
3 3 30 70
3 3 30 71
3 3 30 72
3 3 30 73
3 3 30 74
3 3 30 75
3 3 30 76
3 3 30 77
3 3 30 78
3 3 30 79
3 3 30 80
3 3 30 81
3 3 30 82
3 3 30 83
3 3 30 84
3 3 30 85
3 3 30 86
3 3 30 87
3 3 30 88
3 3 30 89
3 3 30 90
3 3 30 91
3 3 30 92
3 3 30 93
3 3 30 94
3 3 30 95
3 3 30 96
3 3 30 97
3 3 30 98
3 3 30 99
3 3 30 100
3 3 30 101
3 3 30 102
3 3 30 103
3 3 30 104
3 3 30 105
3 3 30 106
3 3 30 107
3 3 30 108
3 3 30 109
3 3 30 110
3 3 30 111
3 3 30 112
3 3 30 113
3 3 30 114
3 3 30 115
3 3 30 116
3 3 30 117
3 3 30 118
3 3 30 119
3 3 30 120
3 3 30 121
3 3 30 122
3 3 30 123
3 3 30 124
3 3 30 125
3 3 30 126
3 3 30 127
3 3 30 128
3 3 30 129
3 3 30 130
3 3 30 131
3 3 30 132
3 3 30 133
3 3 30 134
3 3 30 135
3 3 30 136
3 3 30 137
3 3 30 138
3 3 30 139
3 3 30 140
3 3 30 141
3 3 30 142
3 3 30 143
3 3 30 144
3 3 30 145
3 3 30 146
3 3 30 147
3 3 30 148
3 3 30 149
3 3 35 0
3 3 35 1
3 3 35 2
3 3 35 3
3 3 35 4
3 3 35 5
3 3 35 6
3 3 35 7
3 3 35 8
3 3 35 9
3 3 35 10
3 3 35 11
3 3 35 12
3 3 35 13


4 3 5 128
4 3 5 129
4 3 5 130
4 3 5 131
4 3 5 132
4 3 5 133
4 3 5 134
4 3 5 135
4 3 5 136
4 3 5 137
4 3 5 138
4 3 5 139
4 3 5 140
4 3 5 141
4 3 5 142
4 3 5 143
4 3 5 144
4 3 5 145
4 3 5 146
4 3 5 147
4 3 5 148
4 3 5 149
4 3 10 0
4 3 10 1
4 3 10 2
4 3 10 3
4 3 10 4
4 3 10 5
4 3 10 6
4 3 10 7
4 3 10 8
4 3 10 9
4 3 10 10
4 3 10 11
4 3 10 12
4 3 10 13
4 3 10 14
4 3 10 15
4 3 10 16
4 3 10 17
4 3 10 18
4 3 10 19
4 3 10 20
4 3 10 21
4 3 10 22
4 3 10 23
4 3 10 24
4 3 10 25
4 3 10 26
4 3 10 27
4 3 10 28
4 3 10 29
4 3 10 30
4 3 10 31
4 3 10 32
4 3 10 33
4 3 10 34
4 3 10 35
4 3 10 36
4 3 10 37
4 3 10 38
4 3 10 39
4 3 10 40
4 3 10 41
4 3 10 42
4 3 10 43
4 3 10 44
4 3 10 45
4 3 10 46
4 3 10 47
4 3 10 48
4 3 10 49
4 3 10 50
4 3 10 51
4 3 10 52
4 3 10 53
4 3 10 54
4 3 10 55
4 3 10 56
4 3 10 57
4 3 10 58
4 3 10 59
4 3 10 60
4 3 10 61
4 3 10 62
4 3 10 63
4 3 10 64
4 3 10 65
4 3 10 66
4 3 10 67
4 3 10 68
4 3 10 69
4 3 10 70
4 3 10 71
4 3 10 72
4 3 10 73
4 3 10 74
4 3 10 75
4 3 10 76
4 3 10 77
4 3 10 78


4 3 35 29
4 3 35 30
4 3 35 31
4 3 35 32
4 3 35 33
4 3 35 34
4 3 35 35
4 3 35 36
4 3 35 37
4 3 35 38
4 3 35 39
4 3 35 40
4 3 35 41
4 3 35 42
4 3 35 43
4 3 35 44
4 3 35 45
4 3 35 46
4 3 35 47
4 3 35 48
4 3 35 49
4 3 35 50
4 3 35 51
4 3 35 52
4 3 35 53
4 3 35 54
4 3 35 55
4 3 35 56
4 3 35 57
4 3 35 58
4 3 35 59
4 3 35 60
4 3 35 61
4 3 35 62
4 3 35 63
4 3 35 64
4 3 35 65
4 3 35 66
4 3 35 67
4 3 35 68
4 3 35 69
4 3 35 70
4 3 35 71
4 3 35 72
4 3 35 73
4 3 35 74
4 3 35 75
4 3 35 76
4 3 35 77
4 3 35 78
4 3 35 79
4 3 35 80
4 3 35 81
4 3 35 82
4 3 35 83
4 3 35 84
4 3 35 85
4 3 35 86
4 3 35 87
4 3 35 88
4 3 35 89
4 3 35 90
4 3 35 91
4 3 35 92
4 3 35 93
4 3 35 94
4 3 35 95
4 3 35 96
4 3 35 97
4 3 35 98
4 3 35 99
4 3 35 100
4 3 35 101
4 3 35 102
4 3 35 103
4 3 35 104
4 3 35 105
4 3 35 106
4 3 35 107
4 3 35 108
4 3 35 109
4 3 35 110
4 3 35 111
4 3 35 112
4 3 35 113
4 3 35 114
4 3 35 115
4 3 35 116
4 3 35 117
4 3 35 118
4 3 35 119
4 3 35 120
4 3 35 121
4 3 35 122
4 3 35 123
4 3 35 124
4 3 35 125
4 3 

3 10 149
3 15 0
3 15 1
3 15 2
3 15 3
3 15 4
3 15 5
3 15 6
3 15 7
3 15 8
3 15 9
3 15 10
3 15 11
3 15 12
3 15 13
3 15 14
3 15 15
3 15 16
3 15 17
3 15 18
3 15 19
3 15 20
3 15 21
3 15 22
3 15 23
3 15 24
3 15 25
3 15 26
3 15 27
3 15 28
3 15 29
3 15 30
3 15 31
3 15 32
3 15 33
3 15 34
3 15 35
3 15 36
3 15 37
3 15 38
3 15 39
3 15 40
3 15 41
3 15 42
3 15 43
3 15 44
3 15 45
3 15 46
3 15 47
3 15 48
3 15 49
3 15 50
3 15 51
3 15 52
3 15 53
3 15 54
3 15 55
3 15 56
3 15 57
3 15 58
3 15 59
3 15 60
3 15 61
3 15 62
3 15 63
3 15 64
3 15 65
3 15 66
3 15 67
3 15 68
3 15 69
3 15 70
3 15 71
3 15 72
3 15 73
3 15 74
3 15 75
3 15 76
3 15 77
3 15 78
3 15 79
3 15 80
3 15 81
3 15 82
3 15 83
3 15 84
3 15 85
3 15 86
3 15 87
3 15 88
3 15 89
3 15 90
3 15 91
3 15 92
3 15 93
3 15 94
3 15 95
3 15 96
3 15 97
3 15 98
3 15 99
3 15 100
3 15 101
3 15 102
3 15 103
3 15 104
3 15 105
3 15 106
3 15 107
3 15 108
3 15 109
3 15 110
3 15 111
3 15 112
3 15 113
3 15 114
3 15 115
3 15 116
3 15 117
3 15 118
3 15 119
3 15 120
3 15 121
3 1

3 45 95
3 45 96
3 45 97
3 45 98
3 45 99
3 45 100
3 45 101
3 45 102
3 45 103
3 45 104
3 45 105
3 45 106
3 45 107
3 45 108
3 45 109
3 45 110
3 45 111
3 45 112
3 45 113
3 45 114
3 45 115
3 45 116
3 45 117
3 45 118
3 45 119
3 45 120
3 45 121
3 45 122
3 45 123
3 45 124
3 45 125
3 45 126
3 45 127
3 45 128
3 45 129
3 45 130
3 45 131
3 45 132
3 45 133
3 45 134
3 45 135
3 45 136
3 45 137
3 45 138
3 45 139
3 45 140
3 45 141
3 45 142
3 45 143
3 45 144
3 45 145
3 45 146
3 45 147
3 45 148
3 45 149
3 50 0
3 50 1
3 50 2
3 50 3
3 50 4
3 50 5
3 50 6
3 50 7
3 50 8
3 50 9
3 50 10
3 50 11
3 50 12
3 50 13
3 50 14
3 50 15
3 50 16
3 50 17
3 50 18
3 50 19
3 50 20
3 50 21
3 50 22
3 50 23
3 50 24
3 50 25
3 50 26
3 50 27
3 50 28
3 50 29
3 50 30
3 50 31
3 50 32
3 50 33
3 50 34
3 50 35
3 50 36
3 50 37
3 50 38
3 50 39
3 50 40
3 50 41
3 50 42
3 50 43
3 50 44
3 50 45
3 50 46
3 50 47
3 50 48
3 50 49
3 50 50
3 50 51
3 50 52
3 50 53
3 50 54
3 50 55
3 50 56
3 50 57
3 50 58
3 50 59
3 50 60
3 50 61
3 50 62
3 50 63
3 50 64


KeyboardInterrupt: 

In [ ]:
d = 3
for n in [5]:
    for t in [85 + 5*i for i in range(2)]:
        f = open('output/random_1d_' + str(n) + '_' + str(d) + '_' + str(t), 'a')
        for sample in range(150):
            print(d, t, sample)
            s = random_local_1d(n, d, t)
            f.write(str(sn_pure(s)))
            f.write('\n')
        f.close()  

3 3 55
3 3 60
3 3 65
3 3 70
4 3 55
4 3 60
4 3 65
4 3 70
3 60 0
3 60 1
3 60 2
3 60 3
3 60 4
3 60 5
3 60 6
3 60 7
3 60 8
3 60 9
3 60 10
3 60 11
3 60 12
3 60 13
3 60 14
3 60 15
3 60 16
3 60 17
3 60 18
3 60 19
3 60 20
3 60 21
3 60 22
3 60 23
3 60 24
3 60 25
3 60 26
3 60 27
3 60 28
3 60 29
3 60 30
3 60 31
3 60 32
3 60 33
3 60 34
3 60 35
3 60 36
3 60 37
3 60 38
3 60 39
3 60 40
3 60 41
3 60 42
3 60 43
3 60 44
3 60 45
3 60 46
3 60 47
3 60 48
3 60 49
3 60 50
3 60 51
3 60 52
3 60 53
3 60 54
3 60 55
3 60 56
3 60 57
3 60 58
3 60 59
3 60 60
3 60 61
3 60 62
3 60 63
3 60 64
3 60 65
3 60 66
3 60 67
3 60 68
3 60 69
3 60 70
3 60 71
3 60 72
3 60 73
3 60 74
3 60 75
3 60 76
3 60 77
3 60 78
3 60 79
3 60 80
3 60 81
3 60 82
3 60 83
3 60 84
3 60 85
3 60 86
3 60 87
3 60 88
3 60 89
3 60 90
3 60 91
3 60 92
3 60 93
3 60 94
3 60 95
3 60 96
3 60 97
3 60 98
3 60 99
3 60 100
3 60 101
3 60 102
3 60 103
3 60 104
3 60 105
3 60 106
3 60 107
3 60 108
3 60 109
3 60 110
3 60 111
3 60 112
3 60 113
3 60 114
3 60 115
3 60 116
3